# TODO

- bring in exogenous factors: legislative production, 
-    mapping legal act directory codes - case law directory codes
- bring in exogenous factors: lodgment in year, year-1, year-2,...
- lodgment-judgment-gap; use lodgment-data rather than judgment-data? go for judgment date directly?
- how to deal with ambiguities in dircodes?
- verlängern der timeline? 
- trends auf zweiter dircode ebene

X implement arima model
X implement ols
x missings that are actual 0s

In [ ]:
import pyreadr
import numpy as np 
import pandas as pd
import re 
from itertools import chain # flatten lists

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
tmp = pyreadr.read_r("/Users/david/Documents/david/karriere/projects/predict precedential value/data/data_for_ml_training.RData") 
print(tmp.keys()) # let's check what objects we got
df = tmp["art267_judgments_62_23"] # extract the pandas data frame for object df1
df.head()
df = df.loc[df["directorycodes"] != "no info",]
print(df.isna().sum())
print(df.columns)
print(df.shape)

In [ ]:
df["directorycodes_lower"] = df["directorycodes"].str.lower()
df['substring_column'] = df['directorycodes_lower'].str.slice(0, 4)
df['DoCL_after_Lisbon'] = df['substring_column'].str.contains(r'\d\.\d{2}', regex=True).astype(int)
df['DoCL_after_Lisbon'].value_counts()


df = df.loc[df['DoCL_after_Lisbon'] == 1,]

In [ ]:
list_dircodes_aL = df.loc[df['DoCL_after_Lisbon'] == 1, 'directorycodes_lower'].str.split("@")

#pattern_to_match = re.compile(r"[a-z]-\d\d|\d\.\d\d") # window size=4
pattern_to_match = re.compile(r"[a-z]-|\d\.") # window size=2

window_size = 2

#dir_cod_list_clean = [[item for item in sublist if pattern_to_match.match(item)] for sublist in list_directorycodes]
list_dircodes_aL_num = [
    list(sorted(set([item[0:window_size] for item in sublist if pattern_to_match.match(item[0:window_size])])))
    for sublist in list_dircodes_aL
]

#dir_cod_list_clean = [item for item in sublist for sublist in list_directorycodes]

flattened_list = list(chain(*list_dircodes_aL_num))
#print(len(flattened_list))
print(len(sorted(set(flattened_list))))
# 1311 -> 152
#print(sorted(set(flattened_list)))

pattern = re.compile("[a-z]-\d{2}")
filtered_list = [s for s in flattened_list if pattern.match(s)]
print(len(set(filtered_list)))
# 675 -> 97

pattern = re.compile("\d.\d{2}")
filtered_list = [s for s in flattened_list if pattern.match(s[0:window_size])]
print(len(set(filtered_list)))
# 636 -> 55

In [ ]:
df.loc[df["celex"] == "62014CJ0049", 'date_lodgment'] = '02/03/2014'
df["year_lodgment"] = df["date_lodgment"].str.slice(6,10).astype(int)
df["year_lodgment"].value_counts()

df["year_judgment"] = df["year_judgment"].astype(int)
df["year_judgment"].value_counts()



In [ ]:

new_df = pd.concat([pd.Series(sublist, name="value").astype(float) for sublist in list_dircodes_aL_num], keys=df["year_judgment"]).reset_index(level=0)
print(df.shape)
print(new_df.shape)


In [ ]:
trend_df = new_df.groupby(['year_judgment', 'value']).size().reset_index(name='count')

missing_data = {
    'year_judgment': [2010,2014,2015,2023,2013,2014,2019,2020,2021,2022,2010,2014,2016,2018,2019,2020],
    'value':         [ 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0],
    'count':         [ 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]
    }

trend_df = pd.concat([trend_df, pd.DataFrame(missing_data)])

trend_df_sort = trend_df.sort_values(by=['value', 'year_judgment'])
trend_df_sort
trend_df_sort["value"] = trend_df_sort["value"].astype(str)
trend_df_sort.reset_index(drop=True, inplace=True)
print(trend_df_sort.shape)
trend_df_sort
#trend_df_sort["value"].value_counts()


In [ ]:



trend_df_sort_ols = trend_df_sort
trend_df_sort_ols['count_l1'] = trend_df_sort_ols.groupby(['value'])['count'].shift()
trend_df_sort_ols['count_l2'] = trend_df_sort_ols.groupby(['value'])['count_l1'].shift()
trend_df_sort_ols['count_l3'] = trend_df_sort_ols.groupby(['value'])['count_l2'].shift()
trend_df_sort_ols['count_l4'] = trend_df_sort_ols.groupby(['value'])['count_l3'].shift()
trend_df_sort_ols['count_l5'] = trend_df_sort_ols.groupby(['value'])['count_l4'].shift()

pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
trend_df_sort_ols


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Plot trend lines
sns.lineplot(x='year_judgment', y='count', hue='value', data=trend_df_sort, marker='o')

plt.title('Number of Preliminary Reference Judgments by Year and Directory Code')
plt.xticks(trend_df_sort['year_judgment'].unique(), rotation=45)
plt.xlabel('Year')
plt.ylabel('Number of Judgments')
plt.legend(title='Directory Code')
plt.show()

In [ ]:
# Exogenous variable - assuming 'group' is a categorical variable encoded as integers
exog_data = pd.get_dummies(trend_df_sort['value'], prefix='dircode', drop_first=True)
exog_data.index = trend_df_sort['year_judgment']
exog_data.index
time_series = trend_df_sort.set_index(['year_judgment','value'])['count'].astype(float)#
time_series
exog_data = exog_data.astype(int)

In [ ]:
# limit timeframe
time_series_masked = time_series[time_series.index.get_level_values(0) < 2021]
time_series_masked
time_series_masked.index

In [ ]:
# Set ARIMA order
p = 5  # Autoregressive order
d = 0  # Integration order
q = 0  # Moving average order

time_series_masked_flat = time_series_masked.reset_index(level='value', drop=True)

# Fit SARIMAX model with exogenous variable
model = ARIMA(endog=time_series_masked_flat#, exog=exog_data
              , order=(p, d, q))
#model = SARIMAX(endog=time_series, exog=exog_data, order=(p, d, q))
results = model.fit()


print(results.summary())

In [ ]:
print(results.summary())

In [ ]:
forecast_steps = 10  # Number of steps to predict
forecast = results.get_forecast(steps=forecast_steps)

forecast.predicted_mean
# Extract predicted values
#predicted_values = forecast.predicted_mean
#predicted_values = pd.DataFrame({'predicted': forecast.predicted_mean}, index=time_series_masked.index[-3:])
#predicted_values

In [ ]:
trend_df_sort_ols

In [ ]:
#OLS 
trend_df_sort_ols_train = trend_df_sort_ols.loc[#(trend_df_sort_ols["year_judgment"] >= 2013) & 
    (trend_df_sort_ols["year_judgment"] <= 2020)]
trend_df_sort_ols_train.dropna(inplace=True)
trend_df_sort_ols_train

#model = sm.OLS(trend_df_sort_ols_train['count'], sm.add_constant(trend_df_sort_ols_train[['year_judgment', 'value']])).fit()
#model.summary()


